In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("Info gain.csv")
df

,age,income,student,credit,buy
0,<=30,high,0,fair,0
1,<=30,high,0,excellent,0
2,31...41,high,0,fair,1
3,<40,medium,0,fair,1
4,<40,low,1,fair,1
5,<40,low,1,excellent,0
6,31...41,low,1,excellent,1
7,<=30,medium,0,fair,0
8,<=30,low,1,fair,1
9,<40,medium,1,fair,1


# Information Gain

In [3]:
def CalEntropy(success,fail):
    if(success==0.0 or fail==0.0):
       return 0
    else:
       return (-success*np.log2(success))+(-fail*np.log2(fail)) 

In [4]:
def InfoGain(data,target):
    ones=data[target][data[target]==1] 
    zeros=df[target][df[target]==0]
    probailityofSuccess=len(ones)/(len(ones)+len(zeros))
    probailityOfFail=len(zeros)/(len(ones)+len(zeros))
    entropyofTarget=CalEntropy(probailityofSuccess,probailityOfFail)
    features=data.drop(columns=target)
    for feature in features.columns:
        categories=features[feature].unique()
        totalNumberOfElemntsInFeature=len(features[feature])
        EntropyForCategoryOfFeature=[]
        ProbablilityForCategoryOfFeature=[]
        for category in categories:
            values=features[feature][features[feature]==category]
            probabliltyOfCategory=(len(values)/totalNumberOfElemntsInFeature)
            ProbablilityForCategoryOfFeature.append(probabliltyOfCategory)
            ones=data.loc[data[feature] == category, target][data['buy']==1]
            zeros=data.loc[data[feature] == category, target][data['buy']==0]
            result_of_count={"1":len(ones),"0":len(zeros)}
            successprobability=(result_of_count["1"]/(len(ones)+len(zeros)))
            Failprobability=(result_of_count["0"]/(len(ones)+len(zeros)))
            entropy=CalEntropy(successprobability,Failprobability)
            EntropyForCategoryOfFeature.append(entropy)
            print(f"""
            feature:{feature}
            probablilty Of {category}:{probabliltyOfCategory}
            category:{category}  
            success probability:{successprobability}  
            Fail probability:{Failprobability} 
            entropy:{entropy}""")
        sumEntropyMeaure=0
        for value1,value2 in zip(EntropyForCategoryOfFeature,ProbablilityForCategoryOfFeature):
            entropyMeausre=value1*value2
            sumEntropyMeaure+=entropyMeausre
        print(f"entropy of {feature}:{sumEntropyMeaure.round(3)}")
        print(f"entropyof{target} :{entropyofTarget.round(3)}")
        Gain=entropyofTarget-sumEntropyMeaure
        print(f"Gain of {feature}:{Gain.round(3)}")

In [5]:
InfoGain(df,'buy')


            feature:age
            probablilty Of <=30:0.35714285714285715
            category:<=30  
            success probability:0.4  
            Fail probability:0.6 
            entropy:0.9709505944546686

            feature:age
            probablilty Of 31...41:0.2857142857142857
            category:31...41  
            success probability:1.0  
            Fail probability:0.0 
            entropy:0

            feature:age
            probablilty Of <40:0.35714285714285715
            category:<40  
            success probability:0.6  
            Fail probability:0.4 
            entropy:0.9709505944546686
entropy of age:0.694
entropyofbuy :0.94
Gain of age:0.247

            feature:income
            probablilty Of high:0.2857142857142857
            category:high  
            success probability:0.5  
            Fail probability:0.5 
            entropy:1.0

            feature:income
            probablilty Of medium:0.42857142857142855
            category:me

# naive bayse

In [6]:
def naiveBayse(data, target,listofvalues):
    predictionMap={}
    MapOfProbapilitys={}
    targets=data[target]
    targetvalues=targets.unique().tolist()
    features=data.drop(columns=target)
    featureColumns=features.columns.tolist()
    if(len(listofvalues)!=len(featureColumns)):
        return ValueError("more features than required")
    else:
        targetProbMap=calucalteProbabilityforTarget(targets)
        MapOfProbapilitys.update(targetProbMap)
        featuresProbMap=calulateProbabilityForfeature(features)
        MapOfProbapilitys.update(featuresProbMap)
        jointProbmap=calcualteJointProb(features,targets)
        MapOfProbapilitys.update(jointProbmap)
        for i in targetvalues:
            predictionMap[i]=MapOfProbapilitys[f"target {i}"]
            for j,k in zip(featureColumns,listofvalues):
                predictionMap[i]=predictionMap[i]*MapOfProbapilitys[i][j][k]
        return predictionMap
            
    

In [13]:
naiveBayse(df,'buy',['<40','low',1,'fair'])

{0: 0.0012755102040816328, 1: 0.18596938775510205}

In [10]:
def calucalteProbabilityforTarget(targetdata):
    targetSize=len(targetdata)
    MapOfProbapilitys={}
    targetValues=targetdata.unique().tolist()
    for i in targetValues:
        categoryOfTarget= targetdata[targetdata==i]
        prob=len(categoryOfTarget)/targetSize
        MapOfProbapilitys[f"target {i}"]=prob
    return MapOfProbapilitys

In [11]:
def calulateProbabilityForfeature(features):
    columns=features.columns.tolist()
    MapOfProbapilitys={}
    for i in columns:
        feature=features[i]
        featuresize=len(feature)
        CategoriesOfFeature=feature.unique().tolist()
        MapOfProbapilitys[i]={}
        for j in CategoriesOfFeature:
            categoryOfFeature= feature[feature==j]
            prob=len(categoryOfFeature)/featuresize
            MapOfProbapilitys[i][j]=prob
    return MapOfProbapilitys

In [12]:
def calcualteJointProb(features,target):
    targets=target.unique().tolist()
    columns=features.columns.tolist()
    MapOfProbapilitys={}
    for i in targets:
        MapOfProbapilitys[i]={}
        for j in columns:
            MapOfProbapilitys[i][j]={}
            CategoriesOfFeature=features[j].unique().tolist()
            for k in CategoriesOfFeature:
                categoryOfFeature= features[j][features[j]==k]
                categoryOfFeatureIfTarget=features[j][features[j]==k][target==i]
                prob=len(categoryOfFeatureIfTarget)/len(categoryOfFeature)
                MapOfProbapilitys[i][j][k]=prob
    return MapOfProbapilitys 

In [39]:
def  EuclideanDistance(point1,point2):
    if (type(point1)!=type(tuple()) and type(point2)!=type(tuple())):
        return "please enter variables as tuple"
    elif (len(point1)!=2 and len(point2)!=2):
        return "please enter variables as (x,y)"
    else:
        x=point1[0]-point2[0]
        y=point1[1]-point2[1]
        dis=np.sqrt((x**2)+(y**2))
        return f"distance is {dis}"
        


        
    

In [40]:
EuclideanDistance((2,3),(3,2))

'distance is 1.4142135623730951'

In [53]:
def calculateDistanceBetweenCenterAndGroup(point,group):
    dis={'center':point}
    newcenter=calcualteNewCenterofGroup(group)
    dis['newcenter']=newcenter
    for i in group:
        pointdis=EuclideanDistance(point,i)
        dis[i]=pointdis
    return dis

In [54]:
calculateDistanceBetweenCenterAndGroup((2,3),[(2,3),(3,2),(4,5)])

{'center': (2, 3),
 'newcenter': (3.0, 3.3333333333333335),
 (2, 3): 'distance is 0.0',
 (3, 2): 'distance is 1.4142135623730951',
 (4, 5): 'distance is 2.8284271247461903'}

In [51]:
def calcualteNewCenterofGroup(group):
    x=0
    y=0
    for i in group:
        if (type(i)!=type(tuple())):
            return "please enter variables as tuple"
        elif (len(i)!=2 ):
            return "please enter variables as (x,y)"
        else:
            x=x+i[0]
            y=y+i[1]
    return (x/len(group),y/len(group))
        
           

In [52]:
calcualteNewCenterofGroup([(2,3),(3,2),(4,5)])

(3.0, 3.3333333333333335)